In [1]:
import os
import sys
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment
load_dotenv()

True

In [2]:
from groq import Groq

# Load environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY:
    raise ValueError("❌ GROQ_API_KEY not found in .env file!")

# Configure native Groq client
client = Groq(api_key=GROQ_API_KEY)

model = "llama-3.1-8b-instant"  

print(f"✓ Groq configured successfully")
print(f"✓ Using model: {model}")

✓ Groq configured successfully
✓ Using model: llama-3.1-8b-instant


In [3]:
print("\n" + "="*70)
print("LOADING DATASET")
print("="*70)

df = pd.read_csv('yelp.csv')


LOADING DATASET


In [ ]:
print(f"\nOriginal dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nStar rating distribution:")
print(df['stars'].value_counts().sort_index())

samples_per_star = 40
sample_df = df.groupby('stars', group_keys=False).apply(
    lambda x: x.sample(n=min(samples_per_star, len(x)), random_state=42)
).reset_index(drop=True)

sample_df = sample_df[['text', 'stars']].copy()

print(f"\n✓ Sampled dataset shape: {sample_df.shape}")
print(f"Sampled star distribution:")
print(sample_df['stars'].value_counts().sort_index())
print(f"\nFirst 3 samples:")
print(sample_df.head(3)[['text', 'stars']].to_string())


Original dataset shape: (10000, 10)
Columns: ['business_id', 'date', 'review_id', 'stars', 'text', 'type', 'user_id', 'cool', 'useful', 'funny']

Star rating distribution:
stars
1     749
2     927
3    1461
4    3526
5    3337
Name: count, dtype: int64

✓ Sampled dataset shape: (200, 2)
Sampled star distribution:
stars
1    40
2    40
3    40
4    40
5    40
Name: count, dtype: int64

First 3 samples:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [5]:
def prompt_v1(review_text):
    
    return f"""Classify this Yelp review on a scale of 1-5 stars.

Review: "{review_text}"

Respond with valid JSON only:
{{"predicted_stars": <1-5>, "explanation": "<brief reason>"}}"""



def prompt_v2(review_text):

    return f"""
Rate this Yelp review (1–5):

1★ very negative; major failures; strong dissatisfaction
2★ mostly negative; significant issues; few positives
3★ mixed; clear positives + negatives; neutral tone
4★ mostly positive; minor issues only; satisfied
5★ very positive; enthusiastic praise; no real complaints

EXAMPLES:
1★ → "Food was cold, long wait, rude server." → {{"predicted_stars": 1, "explanation": "Severe complaints"}}
3★ → "Decent burger, soggy fries, friendly service." → {{"predicted_stars": 3, "explanation": "Mixed"}}
4★ → "Loved the pasta, slow check." → {{"predicted_stars": 4, "explanation": "Mostly positive"}}

Review: "{review_text}"

Respond with valid JSON only:
{{"predicted_stars": <1-5>, "explanation": "<brief reason>"}}"""



def prompt_v3(review_text):
    
    return f"""Rate this Yelp review (1-5 stars) by analyzing it systematically.

Review: "{review_text}"

Think through:
1. What specific positive aspects are mentioned?
2. What specific negative aspects are mentioned?
3. What's the overall emotional tone?
4. Are there any strong keywords (love, hate, terrible, amazing)?

Respond with valid JSON only:
{{"predicted_stars": <1-5>, "explanation": "<brief reason>"}}"""




PROMPTS = {
    "V1_Simple": prompt_v1,
    "V2_Detailed_Criteria": prompt_v2,
    "V3_Chain_of_Thought": prompt_v3
}

print("✓ Defined 3 prompt engineering approaches")

✓ Defined 3 prompt engineering approaches


In [ ]:
from pydantic import BaseModel, Field, ValidationError
from typing import Optional

class ReviewPrediction(BaseModel):
    """Schema for validating LLM review rating predictions"""
    predicted_stars: int = Field(..., ge=1, le=5, description="Star rating from 1 to 5")
    explanation: str = Field(..., min_length=1, description="Brief reasoning")

print("✓ Pydantic validation schema defined")

✓ Pydantic validation schema defined


In [7]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, Tuple, Optional
import re
import json
import time

def call_llm(review_text: str, prompt_func: callable, max_retries: int = 3) -> str:
    """Call Groq API with retry logic and error handling."""
    prompt = prompt_func(review_text)
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,  
                max_tokens=500  # Increased - Groq is fast and generous
            )
            return response.choices[0].message.content
        
        except Exception as e:
            error_msg = str(e).lower()
            
            # Groq rate limit handling (much higher limits than other providers)
            if "rate" in error_msg or "limit" in error_msg or "429" in error_msg:
                wait_time = (2 ** attempt) * 3  # 3s, 6s, 12s (faster backoff for Groq)
                print(f"  ⚠ Rate limited. Waiting {wait_time}s... (attempt {attempt + 1}/{max_retries})")
                time.sleep(wait_time)
                continue
            
            # Groq-specific: Model availability issues
            if "model" in error_msg and ("not found" in error_msg or "unavailable" in error_msg):
                print(f"  ❌ Model error: {str(e)[:150]}")
                print(f"     → Check available models at https://console.groq.com/docs/models")
                return f"ERROR: Model not available - {str(e)}"
            
            # Groq-specific: API key issues
            if "unauthorized" in error_msg or "401" in error_msg or "api key" in error_msg:
                print(f"  ❌ Authentication error: {str(e)[:150]}")
                print(f"     → Check API key at https://console.groq.com/keys")
                return f"ERROR: Invalid API key - {str(e)}"
            
            # Generic retry for transient errors
            if attempt < max_retries - 1:
                print(f"  ⚠ Error: {str(e)[:100]}. Retrying in 2s...")
                time.sleep(2)
                continue
            
            # Final failure
            print(f"  ❌ Final attempt failed: {str(e)[:150]}")
            return f"ERROR: {str(e)}"
    
    return "ERROR: Max retries exceeded"


def extract_json_from_response(response_text: str) -> Optional[str]:
    """Extract and clean JSON from LLM response with robust parsing."""
    if not response_text or response_text.startswith('ERROR'):
        return "{}"
    
    # Remove markdown code blocks (Groq sometimes includes these)
    text = response_text.replace('``````', '').strip()
    
    # Find JSON boundaries
    start = text.find('{')
    end = text.rfind('}') + 1
    
    if start == -1 or end <= start:
        return "{}"
    
    json_str = text[start:end]
    
    # Extract predicted_stars with regex
    stars_match = re.search(r'"predicted_stars"\s*:\s*(\d+)', json_str)
    if not stars_match:
        return "{}"
    stars = stars_match.group(1)
    
    # Validate star rating is in range 1-5
    if not (1 <= int(stars) <= 5):
        return "{}"
    
    # Extract explanation (handle multiline and quotes)
    expl_pattern = r'"explanation"\s*:\s*"(.*?)"(?:\s*[,}])'
    expl_match = re.search(expl_pattern, json_str, re.DOTALL)
    
    if expl_match:
        explanation = ' '.join(expl_match.group(1).split())
        # Escape internal quotes
        explanation = explanation.replace('"', '\\"')
    else:
        explanation = "No explanation provided"
    
    # Reconstruct clean JSON
    return f'{{"predicted_stars": {stars}, "explanation": "{explanation}"}}'


def validate_prediction(response_text: str) -> dict:
    """Validate LLM response against the ReviewPrediction schema."""
    result = {
        'valid': False,
        'predicted_stars': None,
        'explanation': None,
        'error': None,
        'raw_response': response_text
    }
    
    try:
        json_str = extract_json_from_response(response_text)
        json_data = json.loads(json_str)
        prediction = ReviewPrediction(**json_data)
        
        result['valid'] = True
        result['predicted_stars'] = prediction.predicted_stars
        result['explanation'] = prediction.explanation
        
    except json.JSONDecodeError as e:
        result['error'] = f"JSON parsing error: {str(e)}"
    except ValidationError as e:
        errors = e.errors()
        error_msgs = [f"{err['loc'][0]}: {err['msg']}" for err in errors]
        result['error'] = f"Validation error: {'; '.join(error_msgs)}"
    except Exception as e:
        result['error'] = f"Unexpected error: {str(e)}"
    
    return result


print("✓ LLM calling and validation functions ready for Groq")


✓ LLM calling and validation functions ready for Groq


In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, Tuple
import time

def call_llm_with_name(review_text: str, prompt_name: str, prompt_func: callable) -> Tuple[str, str]:
    """Call LLM and return (prompt_name, raw_response)"""
    raw_response = call_llm(review_text, prompt_func)
    return (prompt_name, raw_response)


def process_single_review_all_prompts(review_text: str) -> Dict[str, any]:
    """
    Process one review with all 3 prompts simultaneously using threading.
    Optimized for Groq's fast parallel inference.
    """
    output = {}
    
    # Groq handles concurrent requests well - use 3 workers for 3 prompts
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {
            executor.submit(call_llm_with_name, review_text, prompt_name, prompt_func): prompt_name
            for prompt_name, prompt_func in PROMPTS.items()
        }
        
        for future in as_completed(futures):
            prompt_name, raw_response = future.result()
            
            # Determine version prefix
            if 'V1' in prompt_name:
                prefix = 'v1'
            elif 'V2' in prompt_name:
                prefix = 'v2'
            else:  # V3
                prefix = 'v3'
            
            # Validate the response
            validation = validate_prediction(raw_response)
            
            if validation['valid']:
                output[f'{prefix}_predicted_stars'] = validation['predicted_stars']
                output[f'{prefix}_explanation'] = validation['explanation']
            else:
                output[f'{prefix}_predicted_stars'] = None
                output[f'{prefix}_explanation'] = f"ERROR: {validation['error']}"
    
    return output


def process_reviews_with_all_prompts(df: pd.DataFrame, delay: float = 0.5) -> pd.DataFrame:
    """
    Process each review with all 3 prompts simultaneously.
    Optimized for Groq's ultra-fast inference (500+ tokens/sec).
    
    Args:
        df: DataFrame with 'text' and 'stars' columns
        delay: Seconds to wait after processing each review (default 0.5s for Groq)
    """
    result_df = df.copy()
    
    # Initialize all result columns
    result_df['v1_predicted_stars'] = None
    result_df['v1_explanation'] = None
    result_df['v2_predicted_stars'] = None
    result_df['v2_explanation'] = None
    result_df['v3_predicted_stars'] = None
    result_df['v3_explanation'] = None
    
    total_calls = len(df) * 3
    estimated_time = (len(df) * delay + total_calls * 0.5) / 60  # Rough estimate
    
    print(f"\n{'='*70}")
    print(f"PROCESSING WITH GROQ ⚡")
    print(f"{'='*70}")
    print(f"Reviews to process: {len(df)}")
    print(f"Prompts per review: 3 (run in parallel)")
    print(f"Total API calls: {total_calls}")
    print(f"Model: {model}")
    print(f"Estimated time: ~{estimated_time:.1f} minutes")
    print(f"Delay between reviews: {delay}s")
    print(f"{'='*70}\n")
    
    start_time = time.time()
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing Reviews"):
        review_text = row['text']
        
        # Process all 3 prompts in parallel for this review
        results = process_single_review_all_prompts(review_text)
        
        # Store results in DataFrame
        for col, value in results.items():
            result_df.at[idx, col] = value
        
        # Short delay before next review (Groq is fast, minimal delay needed)
        if idx < len(df) - 1:
            time.sleep(delay)
    
    elapsed_time = time.time() - start_time
    
    print(f"\n{'='*70}")
    print(f"✓ Processing complete!")
    print(f"{'='*70}")
    print(f"Total time: {elapsed_time/60:.1f} minutes")
    print(f"Average time per review: {elapsed_time/len(df):.1f} seconds")
    print(f"{'='*70}\n")
    
    return result_df


print("✓ Parallel processing functions ready for Groq")

✓ Parallel processing functions ready for Groq


In [9]:
# %%
# ============================================
# TEST GROQ API CONNECTION
# ============================================
print("\n" + "="*70)
print("TESTING GROQ API CONNECTION")
print("="*70)

test_review = "The food was amazing!"
test_prompt = f"""Rate this review from 1-5 stars:
Review: "{test_review}"
Respond with JSON: {{"predicted_stars": <number>, "explanation": "<reason>"}}"""

try:
    test_response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": test_prompt}],
        temperature=0,
        max_tokens=200
    )
    print(f"✓ Groq API working!")
    print(f"Response: {test_response.choices[0].message.content}")
    
    # Additional validation
    validation = validate_prediction(test_response.choices[0].message.content)
    if validation['valid']:
        print(f"✓ Valid JSON response: {validation['predicted_stars']}★")
        print(f"  Explanation: {validation['explanation']}")
    else:
        print(f"⚠ Response format issue: {validation['error']}")
        
except Exception as e:
    print(f"❌ Groq API test failed: {e}")
    print("Check your API key and internet connection")
    print("Get your free API key at: https://console.groq.com/keys")



TESTING GROQ API CONNECTION
✓ Groq API working!
Response: {"predicted_stars": 5, "explanation": "The review is extremely positive, using the superlative 'amazing' to describe the food, indicating a perfect or near-perfect experience."}
✓ Valid JSON response: 5★
  Explanation: The review is extremely positive, using the superlative 'amazing' to describe the food, indicating a perfect or near-perfect experience.


In [10]:
# %%
# ============================================
# STARTING EVALUATION WITH GROQ
# ============================================
print("\n" + "="*70)
print("STARTING EVALUATION WITH GROQ ⚡")
print("="*70)

# ✓ Optimized for Groq's ultra-fast inference
predictions_df = process_reviews_with_all_prompts(sample_df, delay=0.5)

# Check success rates
print("\n" + "="*70)
print("JSON VALIDITY RATES:")
print("="*70)

validity_results = []
for version in ['v1', 'v2', 'v3']:
    success = predictions_df[f'{version}_predicted_stars'].notna().sum()
    total = len(sample_df)
    percentage = (success/total*100)
    validity_results.append(percentage)
    print(f"{version.upper()}: {success}/{total} ({percentage:.1f}%)")

# Overall validity
avg_validity = sum(validity_results) / len(validity_results)
print(f"\nAVERAGE VALIDITY: {avg_validity:.1f}%")

if avg_validity < 80:
    print("⚠ Low validity rate - check prompt formatting or model compatibility")
elif avg_validity < 95:
    print("✓ Good validity rate - minor improvements possible")
else:
    print("✓ Excellent validity rate!")

# Sample predictions comparison
print("\n" + "="*70)
print("SAMPLE PREDICTIONS (First 10 Reviews):")
print("="*70)
display_cols = ['stars', 'v1_predicted_stars', 'v2_predicted_stars', 'v3_predicted_stars']
print(predictions_df[display_cols].head(10).to_string(index=True))

# Quick accuracy check (for valid predictions only)
print("\n" + "="*70)
print("QUICK ACCURACY CHECK:")
print("="*70)

for version in ['v1', 'v2', 'v3']:
    valid_mask = predictions_df[f'{version}_predicted_stars'].notna()
    if valid_mask.sum() > 0:
        valid_df = predictions_df[valid_mask]
        accuracy = (valid_df['stars'] == valid_df[f'{version}_predicted_stars']).sum() / len(valid_df) * 100
        print(f"{version.upper()}: {accuracy:.1f}% accuracy ({valid_mask.sum()} valid samples)")
    else:
        print(f"{version.upper()}: No valid predictions")



STARTING EVALUATION WITH GROQ ⚡

PROCESSING WITH GROQ ⚡
Reviews to process: 200
Prompts per review: 3 (run in parallel)
Total API calls: 600
Model: llama-3.1-8b-instant
Estimated time: ~6.7 minutes
Delay between reviews: 0.5s



Processing Reviews:   2%|▎         | 5/200 [00:03<02:20,  1.39it/s]

  ⚠ Rate limited. Waiting 3s... (attempt 1/3)


Processing Reviews:   4%|▎         | 7/200 [00:42<31:35,  9.82s/it]

  ⚠ Rate limited. Waiting 3s... (attempt 1/3)


...

  ⚠ Rate limited. Waiting 3s... (attempt 1/3)


Processing Reviews: 100%|█████████▉| 199/200 [34:52<00:09,  9.11s/it]

  ⚠ Rate limited. Waiting 3s... (attempt 1/3)


Processing Reviews: 100%|██████████| 200/200 [35:05<00:00, 10.53s/it]


✓ Processing complete!
Total time: 35.1 minutes
Average time per review: 10.5 seconds


JSON VALIDITY RATES:
V1: 200/200 (100.0%)
V2: 200/200 (100.0%)
V3: 200/200 (100.0%)

AVERAGE VALIDITY: 100.0%
✓ Excellent validity rate!

SAMPLE PREDICTIONS (First 10 Reviews):
   stars v1_predicted_stars v2_predicted_stars v3_predicted_stars
0      1                  1                  1                  1
1      1                  1                  1                  1
2      1                  1                  1                  1
3      1                  1                  1                  1
4      1                  1                  1                  1
5      1                  1                  1                  1
6      1                  1                  1                  1
7      1                  1                  1                  1
8      1                  1                  2                  2
9      1                  1                  1                  1

QUICK A

In [11]:
print("\n" + "="*70)
print("EVALUATION METRICS")
print("="*70)

results = []
for version, name in [('v1', 'V1_Simple'), ('v2', 'V2_Detailed_Criteria'), 
                      ('v3', 'V3_Chain_of_Thought')]:
    
    valid_mask = predictions_df[f'{version}_predicted_stars'].notna()
    valid_df = predictions_df[valid_mask].copy()
    
    if len(valid_df) > 0:
        actual = valid_df['stars'].astype(int)
        predicted = valid_df[f'{version}_predicted_stars'].astype(int)
        
        accuracy = (actual == predicted).sum() / len(valid_df) * 100
        mae = mean_absolute_error(actual, predicted)
        
        diff = abs(actual - predicted)
        exact = (diff == 0).sum()
        off_1 = (diff == 1).sum()
        off_2_plus = (diff >= 2).sum()
        
        validity = len(valid_df) / len(predictions_df) * 100
        
        results.append({
            'Prompt': name,
            'Accuracy (%)': round(accuracy, 2),
            'MAE': round(mae, 3),
            'Validity (%)': round(validity, 1),
            'Exact Match': exact,
            'Off by 1': off_1,
            'Off by 2+': off_2_plus,
            'Valid Samples': len(valid_df)
        })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# Winner
if len(results_df) > 0:
    best_idx = results_df['Accuracy (%)'].idxmax()
    best = results_df.loc[best_idx]
    
    print("\n" + "="*70)
    print("🏆 BEST PERFORMING PROMPT")
    print("="*70)
    print(f"Winner: {best['Prompt']}")
    print(f"  Accuracy: {best['Accuracy (%)']}%")
    print(f"  MAE: {best['MAE']}")
    print(f"  Validity: {best['Validity (%)']}%")
    print(f"  Exact Matches: {best['Exact Match']}/{best['Valid Samples']}")


EVALUATION METRICS
              Prompt  Accuracy (%)   MAE  Validity (%)  Exact Match  Off by 1  Off by 2+  Valid Samples
           V1_Simple          67.0 0.350         100.0          134        64          2            200
V2_Detailed_Criteria          67.0 0.350         100.0          134        64          2            200
 V3_Chain_of_Thought          68.5 0.335         100.0          137        61          2            200

🏆 BEST PERFORMING PROMPT
Winner: V3_Chain_of_Thought
  Accuracy: 68.5%
  MAE: 0.335
  Validity: 100.0%
  Exact Matches: 137/200


In [12]:
results_df.to_csv('prompt_evaluation_metrics.csv', index=False)

print("\n" + "="*70)
print("RESULTS SAVED")
print("="*70)


RESULTS SAVED


In [13]:
print("\n" + "="*70)
print("ERROR ANALYSIS: PREDICTION DISCREPANCIES")
print("="*70)

def analyze_prediction_errors(df, version='v3', version_name='V3_Chain_of_Thought'):
    """Analyze prediction errors for a specific prompt version"""
    
    valid_mask = df[f'{version}_predicted_stars'].notna()
    analysis_df = df[valid_mask].copy()
    
    if len(analysis_df) == 0:
        print(f"\n{version_name}: No valid predictions to analyze")
        return {'off_by_1': pd.DataFrame(), 'off_by_2_plus': pd.DataFrame(), 'analysis_df': pd.DataFrame()}
    
    analysis_df['error'] = abs(analysis_df['stars'] - analysis_df[f'{version}_predicted_stars'])
    analysis_df['error_type'] = analysis_df['error'].apply(
        lambda x: 'Exact Match' if x == 0 else ('Off by 1' if x == 1 else 'Off by 2+')
    )
    
    print(f"\n{'='*80}")
    print(f"ANALYSIS FOR: {version_name}")
    print(f"{'='*80}")
    
    print(f"\nError Distribution:")
    print(analysis_df['error_type'].value_counts().sort_index())
    
    off_by_1 = analysis_df[analysis_df['error'] == 1].copy()
    off_by_2_plus = analysis_df[analysis_df['error'] >= 2].copy()
    
    print(f"\n{'-'*80}")
    print(f"OFF BY 1 CASES ({len(off_by_1)} total)")
    print(f"{'-'*80}")
    
    if len(off_by_1) > 0:
        for idx, row in off_by_1.head(5).iterrows():
            actual = int(row['stars'])
            predicted = int(row[f'{version}_predicted_stars'])
            direction = "↑" if predicted > actual else "↓"
            
            print(f"\nExample {idx}:")
            print(f"  Actual: {actual}★  |  Predicted: {predicted}★ {direction}")
            print(f"  Review: {row['text'][:150]}...")
            print(f"  Explanation: {row[f'{version}_explanation']}")
    
    print(f"\n{'-'*80}")
    print(f"OFF BY 2+ CASES ({len(off_by_2_plus)} total) - MAJOR ERRORS")
    print(f"{'-'*80}")
    
    if len(off_by_2_plus) > 0:
        for idx, row in off_by_2_plus.iterrows():
            actual = int(row['stars'])
            predicted = int(row[f'{version}_predicted_stars'])
            error = int(row['error'])
            direction = "↑" if predicted > actual else "↓"
            
            print(f"\nExample {idx} [ERROR: {error}]:")
            print(f"  Actual: {actual}★  |  Predicted: {predicted}★ {direction}")
            print(f"  Review: {row['text'][:200]}...")
            print(f"  Explanation: {row[f'{version}_explanation']}")
    
    return {
        'off_by_1': off_by_1,
        'off_by_2_plus': off_by_2_plus,
        'analysis_df': analysis_df
    }

# Analyze each version
error_results = {}
for version, name in [('v1', 'V1_Simple'), 
                      ('v2', 'V2_Detailed_Criteria'), 
                      ('v3', 'V3_Chain_of_Thought')]:
    error_results[version] = analyze_prediction_errors(predictions_df, version, name)

print(f"\n{'='*80}")
print("ERROR ANALYSIS COMPLETE")
print(f"{'='*80}")


ERROR ANALYSIS: PREDICTION DISCREPANCIES

ANALYSIS FOR: V1_Simple

Error Distribution:
error_type
Exact Match    134
Off by 1        64
Off by 2+        2
Name: count, dtype: int64

--------------------------------------------------------------------------------
OFF BY 1 CASES (64 total)
--------------------------------------------------------------------------------

Example 13:
  Actual: 1★  |  Predicted: 2★ ↑
  Review: I love Saddle Ranch for its casual night time festivities (including watching people ride the mechanical bull hehehe), but I cannot ignore the fact th...
  Explanation: The reviewer enjoyed the atmosphere and unique desserts, but was extremely disappointed with the food quality, describing it as 'disgusting', 'dry', and 'tasteless'.

Example 24:
  Actual: 1★  |  Predicted: 2★ ↑
  Review: I went today to meet my daughter for lunch.  We are both big sushi enthusiasts.  The interior and atmosphere was beautiful.  Well appointed.
The servi...
  Explanation: The reviewer 